In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# DOT 파일 경로
dot_file_path = "/content/drive/MyDrive/1-cpg.dot"

# DOT 파일 읽기
with open(dot_file_path, "r") as dot_file:
    lines = dot_file.readlines()

# 디버깅 메시지: DOT 파일 내용 출력
print("DOT File Contents:")
for line in lines:
    print(line.strip())

In [ ]:
!pip install node2vec

In [ ]:
import numpy as np
from gensim.models import Word2Vec
import networkx as nx
from random import shuffle
from node2vec import Node2Vec  # Node2Vec 모듈 가져오기
# 노드 정보 추출
is_node = True
nodes = {}

for line in lines:
    line = line.strip()
    if line.startswith('digraph'):
        continue  # 그래프 시작 라인 무시
    if '->' in line:
        is_node = False
        continue
    if line == '}':
        break  # 그래프 정의 끝나면 종료
    if is_node:
        parts = line.split('[label = ')
        node_id = parts[0].strip('" ')
        label = parts[1].strip('" ]')
        nodes[node_id] = label
    elif line.endswith('{'):
        is_node = True

# "IDENTIFIER" 또는 "METHOD" 레이블을 가진 노드 추출
matching_nodes = {node_id: label for node_id, label in nodes.items() if "CONTROL_STRUCTURE" in label or "IDENTIFIER" in label or "LITERAL" in label or "LOCAL" in label or "operator" in label}
print(matching_nodes)
# 그래프 생성 및 노드 추가
G = nx.Graph()

for node_id, label in matching_nodes.items():
    G.add_node(node_id, label=label)

# Node2Vec를 사용하여 노드 임베딩 생성
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

# 임베딩 데이터 추출
node_embeddings = np.array([model.wv[node_id] for node_id in matching_nodes.keys()])
print(node_embeddings)

In [ ]:
# DOT 파일을 읽고 CFG 및 DDG 엣지 정보를 저장할 자료구조 초기화
cfg_edges = []
ddg_edges = []
cdg_edges = []

for line in lines:
    line = line.strip()
    if line.startswith('digraph'):
        continue  # 그래프 시작 라인 무시
    if '->' in line:
        parts = line.split('[ label = "')
        if len(parts) > 1:
            edge_labels = parts[1].split('"]')
            edge_type = edge_labels[0].strip()
            if edge_type.startswith("CFG:") or edge_type.startswith("DDG:") or edge_type.startswith("CDG:"):
                # 엣지 정보 추출
                source, target = map(str.strip, parts[0].split('->'))
                if edge_type.startswith("CFG:"):
                    cfg_edges.append((source, target))
                elif edge_type.startswith("DDG:"):
                    ddg_edges.append((source, target))
                else:
                    cdg_edges.append((source, target))

# "CFG:" 레이블을 가진 CFG 엣지 출력
print("CFG Edges:")
for edge in cfg_edges:
    print(edge)

# "DDG:" 레이블을 가진 DDG 엣지 출력
print("\nDDG Edges:")
for edge in ddg_edges:
    print(edge)

# "CDG:" 레이블을 가진 CDG 엣지 출력
print("\nCDG Edges:")
for edge in cdg_edges:
    print(edge)




In [ ]:
from gensim.models import Word2Vec

# 엣지 임베딩 생성 함수
def generate_edge_embeddings(edge_list, dimensions=64, walk_length=30, num_walks=200, workers=4):
    # 엣지 정보를 담은 리스트에서 중복된 엣지 제거
    unique_edges = list(set(edge_list))

    # 엣지를 문자열로 변환
    edge_strings = [f"{edge[0]} -> {edge[1]}" for edge in unique_edges]

    # 엣지 임베딩을 위한 모델 생성
    model = Word2Vec(sentences=edge_strings, vector_size=dimensions, window=10, min_count=1, sg=1, workers=workers)

    # 모델 학습
    model.train(edge_strings, total_examples=len(edge_strings), epochs=10)

    return model

# CFG 엣지 임베딩 생성
cfg_edge_model = generate_edge_embeddings(cfg_edges)
print(cfg_edge_model)
print(cfg_edge_model.wv)


In [ ]:
vector_keys = cfg_edge_model.wv.index_to_key
print("벡터 키 목록:", vector_keys)

In [ ]:
# DOT 파일을 읽고 CFG 및 DDG 엣지 정보를 저장할 자료구조 초기화
cfg_edges = []
ddg_edges = []

# 엣지 정보 추출
for line in lines:
    line = line.strip()
    if line.startswith('digraph'):
        continue  # 그래프 시작 라인 무시
    if '->' in line:
        parts = line.split('[ label = "')
        if len(parts) > 1:
            edge_labels = parts[1].split('"]')
            edge_type = edge_labels[0].strip()
            if edge_type.startswith("CFG:") or edge_type.startswith("DDG:"):
                if edge_type.startswith("CFG:"):
                    cfg_edges.append(line)
                else:
                    ddg_edges.append(line)

# "CFG:" 레이블을 가진 CFG 엣지 출력
print("CFG Edges:")
for edge in cfg_edges:
    print(edge)

# "DDG:" 레이블을 가진 DDG 엣지 출력
print("\nDDG Edges:")
for edge in ddg_edges:
    print(edge)

In [ ]:
from gensim.models import Word2Vec

# 엣지 정보 추출
edges = []
for edge_str in cfg_edges:
    parts = edge_str.split(" -> ")
    source = parts[0].strip()
    target = parts[1].split("  [")[0].strip()
    edges.append([source, target])  # 엣지를 리스트로 변환

# Word2Vec 모델 학습
model = Word2Vec(sentences=edges, vector_size=64, window=5, sg=1, min_count=1, workers=4)

# 학습된 모델을 저장
model.save("edge_embedding.model")

# 저장된 모델을 로드
loaded_model = Word2Vec.load("edge_embedding.model")

In [ ]:
vector_keys = loaded_model.wv.index_to_key
print("벡터 키 목록:", vector_keys)

# 엣지 임베딩 확인 (예시: "20 -> 30"의 엣지 임베딩)
# edge_embedding = loaded_model.wv[[ '"20"']]  # 출발 노드와 도착 노드를 리스트로 전달
# print("Edge embedding (20 -> 30):", edge_embedding)